In [1]:
from robust_motifs.data import import_connectivity_matrix, save_count_graph_from_matrix, MPDataManager
from pathlib import Path

In [2]:
from robust_motifs.counting import get_extended_simplices_dense

In [3]:
import multiprocessing as mp

In [4]:
import time

In [5]:
from tqdm import tqdm

In [6]:
import numpy as np

In [7]:
import matplotlib.pyplot as plt

In [8]:
import sys

In [9]:
pool = mp.Pool()
m = import_connectivity_matrix(dataframe = False, type = 'csr')

100%|██████████| 55/55 [00:12<00:00,  4.54it/s]


In [10]:
save_path = Path("data/temp/test/list/conn.flag")
manager = MPDataManager(save_path, m)

In [11]:
for dimension in range(1,7):
    array = np.empty(shape = (manager._count_file['Cells_' + str(dimension)].shape[0]*15,),dtype = np.int16,)
    indptr = np.empty(shape = (manager._count_file['Cells_' + str(dimension)].shape[0]+1,),dtype = np.int32,)
    indptr[0] = 0
    count = 0
    with tqdm() as pbar:
        for iterator in manager.mp_chunks(dimension, 10**7):
            r = pool.imap(get_extended_simplices_dense, iterator, chunksize = 5000)
            for element in r:
                count += 1
                indptr[count] = indptr[count-1] + len(element)
                array[indptr[count-1]:indptr[count]] = element
                pbar.update()
    path = Path("data/temp/test/list/esd"+str(dimension)+".npz")
    np.savez_compressed(open(path, 'wb'), array[:indptr[-1]])
    path = Path("data/temp/test/list/esd"+str(dimension)+"indptr.npz")
    np.savez_compressed(open(path, 'wb'), indptr)
    del array
    del indptr

7648079it [02:11, 58328.20it/s] 
73036616it [21:41, 56111.24it/s] 
59945205it [17:44, 56315.22it/s] 
6599529it [01:55, 57105.79it/s] 
133115it [00:04, 28319.07it/s]
529it [00:02, 254.52it/s]
